In [2]:
import os
import re
import random
import time

In [3]:
import cloudscraper
from bs4 import BeautifulSoup
import geopandas as gpd
import pandas as pd

In [4]:
from shapely.geometry import Point
from tqdm.notebook import tqdm

## Get State Data

In [5]:
file_path = "data/cb_2018_us_state_500k/cb_2018_us_state_500k.shp"
states_df = gpd.read_file(file_path)

In [6]:
states_df = states_df[["STUSPS", "NAME", "geometry"]]

## Get Wendy's Data

In [7]:
scraper = cloudscraper.create_scraper()

In [8]:
def get_coords(store_url: str) -> Point:
    r = scraper.get(store_url)
    soup = BeautifulSoup(r.text, "html.parser")
    lat = float(soup.find("meta", {"itemprop": "latitude"}).attrs["content"])
    lon = float(soup.find("meta", {"itemprop": "longitude"}).attrs["content"])
    return Point(lon, lat)

In [40]:
r = scraper.get("https://locations.wendys.com/united-states")
soup = BeautifulSoup(r.text, "html.parser")
state_as = soup.find_all("a", {"class": "Directory-listLink"})[:-2]

# Parse all locations in a state
for state_a in tqdm(state_as, desc="Parsing States"):
    state_store_list = []
    state_code = state_a["href"].split("/")[1].upper()
    state_url = os.path.join("https://locations.wendys.com", state_a["href"])

    # If File exists continue
    if os.path.isfile(f"data/states/{state_code}.gpkg"):
        continue

    r_state = scraper.get(state_url)

    if r_state.status_code != 200:
        print(state_code)
        continue

    soup_state = BeautifulSoup(r_state.text, "html.parser")
    city_as = soup_state.find_all("a", {"class": "Directory-listLink"})[:-2]
    city_urls = [
        "/".join(city_a["href"].split("/")[:4]).replace(
            "..", "https://locations.wendys.com"
        )
        for city_a in city_as
    ]

    if state_code == "DC":
        city_urls = [state_url]

    for city_url in tqdm(city_urls, desc=f"Parsing Locations in {state_code}"):
        r_city = scraper.get(city_url)
        soup_city = BeautifulSoup(r_city.text, "html.parser")
        city_as = soup_city.find_all("a", {"data-ya-track": "visitpage"})
        restaurant_urls = [
            city_a["href"].replace("../..", "https://locations.wendys.com")
            for city_a in city_as
        ]
        for restaurant_url in restaurant_urls:
            try:
                point = get_coords(restaurant_url)
            except Exception as e:
                print(restaurant_url, e)
            store_dict = {"STUSPS": state_code, "geometry": point}
            state_store_list.append(store_dict)
            time.sleep(random.uniform(0.01, 0.5))

    if state_store_list:
        print(len(state_store_list), "stores")
        state_dennys_gdf = gpd.GeoDataFrame(state_store_list, crs=4326)
        state_dennys_gdf.to_file(f"data/states/{state_code}.gpkg")

Parsing States:   0%|          | 0/51 [00:00<?, ?it/s]

Parsing Locations in DC:   0%|          | 0/1 [00:00<?, ?it/s]

3 stores


In [41]:
wendys_state_gdfs = []
states_path = "data/states"
for file in os.listdir("data/states"):
    constructed_path = os.path.join(states_path, file)
    if constructed_path.endswith(".gpkg"):
        wendys_state_gdf = gpd.read_file(constructed_path)
        wendys_state_gdfs.append(wendys_state_gdf)

In [42]:
wendys_gdf = gpd.GeoDataFrame(pd.concat(wendys_state_gdfs, ignore_index=True))

In [43]:
wendys_gdf = wendys_gdf.to_crs(9311)
wendys_gdf.to_file(f"data/wendys.gpkg")

## Get Population Data

In [44]:
state_populations = pd.read_excel(
    "data/NST-EST2024-POP.xlsx", sheet_name=None, engine="openpyxl"
)

In [45]:
state_populations_df = state_populations["NST-EST2024-POP"][
    [
        "table with row headers in column A and column headers in rows 3 through 4. (leading dots indicate sub-parts)",
        "Unnamed: 5",
    ]
]
state_populations_df = state_populations_df.rename(
    columns={
        "table with row headers in column A and column headers in rows 3 through 4. (leading dots indicate sub-parts)": "NAME",
        "Unnamed: 5": "POPULATION",
    }
)
state_populations_df["NAME"] = state_populations_df["NAME"].str[1:]

In [46]:
states_with_population_df = states_df.merge(state_populations_df, on="NAME", how="left")
states_with_population_df = states_with_population_df[
    ["STUSPS", "NAME", "POPULATION", "geometry"]
]

In [47]:
states_with_population_df = states_with_population_df.dropna()

## Merge Data

In [48]:
wendys_state_counts_df = pd.DataFrame(
    wendys_gdf.groupby("STUSPS").size(), columns=["WENDYS"]
).reset_index()

In [49]:
wendys_state_counts_df = states_with_population_df.merge(
    wendys_state_counts_df, on="STUSPS", how="left"
)
wendys_state_counts_df = wendys_state_counts_df.fillna(0)

In [50]:
wendys_state_counts_df["per_1000"] = wendys_state_counts_df["WENDYS"] / (
    wendys_state_counts_df["POPULATION"] / 1000
)
wendys_state_counts_df["per_10k"] = wendys_state_counts_df["WENDYS"] / (
    wendys_state_counts_df["POPULATION"] / 10_000
)
wendys_state_counts_df["per_100k"] = wendys_state_counts_df["WENDYS"] / (
    wendys_state_counts_df["POPULATION"] / 100000
)
wendys_state_counts_df["per_500k"] = wendys_state_counts_df["WENDYS"] / (
    wendys_state_counts_df["POPULATION"] / 500_000
)
wendys_state_counts_df["per_1m"] = wendys_state_counts_df["WENDYS"] / (
    wendys_state_counts_df["POPULATION"] / 1_000_000
)
wendys_state_counts_df["per_capita"] = (
    wendys_state_counts_df["POPULATION"] / wendys_state_counts_df["WENDYS"]
)

In [51]:
wendys_state_counts_df = wendys_state_counts_df.to_crs(9311)

In [52]:
wendys_state_counts_df.to_file("data/wendys_per_state.gpkg")